In [10]:
# Importações de bibliotecas

import pandas as pd

In [11]:
# Leitura de arquivos

covid19CleanComplete = pd.read_csv("Arquivos csv/covid_19_clean_complete.csv")
worldOMeterData = pd.read_csv("Arquivos csv/worldometer_data.csv")

In [12]:
# Funções

def verificarProvinceState(linha):
    """"É  uma função que recebe uma linha do banco de dados como argumento e, se a coluna "Province/State" não
    for valor faltante, seu valor será concatenado a com coluna "Country/Region"""
    
    if pd.notna(linha['Province/State']) == True:
        linha['Country/Region'] = linha['Country/Region'] + "_" + linha[
            'Province/State']
        return linha
    return linha

In [13]:
# Questão 01
# Letra a - Verificando a quantidade de linhas e colunas
print("O arquivo tem {} linhas e {} colunas".format(covid19CleanComplete.shape[0], covid19CleanComplete.shape[1]))

# Letra b e c - Verificando os nomes das colunas e qual o tipo de dado de cada uma
print("\nAs colunas são:")
for columnsName in covid19CleanComplete.columns:
    print("- {}, Tipo de dado: {}".format(columnsName, covid19CleanComplete[columnsName].dtype))
    
# Letra d - Transformando a coluna Date do tipo "object" para o tipo "datetime64"
covid19CleanComplete["Date"] = pd.to_datetime(covid19CleanComplete["Date"], format="%Y-%m-%d")
print("\nO tipo de dado da coluna date foi alterado para", covid19CleanComplete["Date"].dtype)

# Letra e - Estatística sobre o banco de dados
# Exibindo a média e número máximo de casos confirmados no Mundo
print("\nNúmero médio de casos confirmados: {:.2f}".format(covid19CleanComplete.Confirmed.describe().loc['mean']))
print("Máximo de casos confirmados em um dia:", covid19CleanComplete.Confirmed.describe().loc['max'])

# Exibindo a média e número máximo de mortes no Mundo
print("\nNúmero médio de mortes confirmados: {:.2f}".format(covid19CleanComplete.Deaths.describe().loc['mean']))
print("Máximo de mortes confirmados em um dia:", covid19CleanComplete.Deaths.describe().loc['max'])

# Exibindo número minimo e máximo para cada país
print("\n", covid19CleanComplete.groupby(['Country/Region']).Confirmed.agg([min, max]))

# Letra f - Se necessário, transformar colunas pertinentes para formato de data
# Não tinha nenhuma outra coluna de data, apenas a que já foi transformada no item d.

# Letra g - Colunas que apresentam NaN
print("\nAs colunas que apresentam NaN são:")
for columnsName in covid19CleanComplete.columns:
    if pd.isnull(covid19CleanComplete[columnsName])[1] == True:
        print("- {}".format(columnsName))

O arquivo tem 49068 linhas e 10 colunas

As colunas são:
- Province/State, Tipo de dado: object
- Country/Region, Tipo de dado: object
- Lat, Tipo de dado: float64
- Long, Tipo de dado: float64
- Date, Tipo de dado: object
- Confirmed, Tipo de dado: int64
- Deaths, Tipo de dado: int64
- Recovered, Tipo de dado: int64
- Active, Tipo de dado: int64
- WHO Region, Tipo de dado: object

O tipo de dado da coluna date foi alterado para datetime64[ns]

Número médio de casos confirmados: 16884.90
Máximo de casos confirmados em um dia: 4290259.0

Número médio de mortes confirmados: 884.18
Máximo de mortes confirmados em um dia: 148011.0

                     min    max
Country/Region                
Afghanistan           0  36263
Albania               0   4880
Algeria               0  27973
Andorra               0    907
Angola                0    950
...                 ...    ...
West Bank and Gaza    0  10621
Western Sahara        0     10
Yemen                 0   1691
Zambia                

In [14]:
# Questão 02
# Letra a, b, c e d - Nome das províncias chinesas contida no dataset e seus dados "Confirmed", "Active", "Deaths", "Recovered"
# Separando apenas as provincias da China
provinceChina = covid19CleanComplete.groupby(['Country/Region', 'Province/State'])[['Confirmed','Active', 'Deaths','Recovered'
                                                             ]].max()
# Nesse ponto provinceChina é do tipo multi-index, logo é necessário usar o 
# reset_index() para converter de volta ao index regular
provinceChina = provinceChina.reset_index().groupby(['Country/Region']).get_group('China')

# Tirando as pronvíncias/estados duplicados
provinceChina = provinceChina.drop_duplicates(subset = ['Province/State'])

# Letra e - Produza um DataFrame contendo as 5 regiões com maior número de
# casos confirmados
provinceChina = provinceChina.sort_values(by = 'Confirmed', ascending = False
                                          ).head()
provinceChina = provinceChina.reset_index()
print("\n As 5 províncias da China que mais registraram casos foram:")
print(provinceChina['Province/State'])


 As 5 províncias da China que mais registraram casos foram:
0        Hubei
1    Hong Kong
2    Guangdong
3        Henan
4     Zhejiang
Name: Province/State, dtype: object


In [15]:
# Questão 03
# Letra a - Concatenar o valor da coluna "Province/State" com coluna "Country/Region"

# Letra b - Faça uma cópia do banco de dados para poder retirar informações sem perder o banco de dados original
# Letra c - Aplique, por meio da método "apply", a função criada no item a
newCovid19CleanComplete = covid19CleanComplete.apply(verificarProvinceState, axis=1)

# Letra d - Exclua a coluna "Province/State" do novo dataframe
newCovid19CleanComplete = newCovid19CleanComplete.drop(["Province/State"], axis = 1)
print(newCovid19CleanComplete)

              Country/Region        Lat       Long       Date  Confirmed  \
0                Afghanistan  33.939110  67.709953 2020-01-22          0   
1                    Albania  41.153300  20.168300 2020-01-22          0   
2                    Algeria  28.033900   1.659600 2020-01-22          0   
3                    Andorra  42.506300   1.521800 2020-01-22          0   
4                     Angola -11.202700  17.873900 2020-01-22          0   
...                      ...        ...        ...        ...        ...   
49063  Sao Tome and Principe   0.186400   6.613100 2020-07-27        865   
49064                  Yemen  15.552727  48.516388 2020-07-27       1691   
49065                Comoros -11.645500  43.333300 2020-07-27        354   
49066             Tajikistan  38.861000  71.276100 2020-07-27       7235   
49067                Lesotho -29.610000  28.233600 2020-07-27        505   

       Deaths  Recovered  Active             WHO Region  
0           0          0     

In [16]:
# Questão 04
# Letra a - Importar o banco de dados "wolrd meter data.csv"

# Letra b - Tome apenas as infromações de 'Population', 'Country/Region', 'Continent' do banco de dados do item a
worldOMeterData = worldOMeterData[['Population', 'Country/Region', 'Continent']]

# Letra c - Agrupe o "covid_19_clean_complete" por países.
# A função de agregação foi apenas o max das mortes pois é o único dado que importa na questão
covid19CleanComplete_country = covid19CleanComplete.groupby(['Country/Region'])[
    ['Deaths']].max()
covid19CleanComplete_country = covid19CleanComplete_country.reset_index()

# Letra d - Mescle os dois dataframe, a partir dos países, contendo apenas as informações de mortes, população e continente
covid19_world = pd.merge(covid19CleanComplete_country, worldOMeterData, how = 'outer')

# Letra e - Agrupe o novo dataset agora pelo continente
covid19_world_continent = covid19_world.groupby(['Continent'])[['Deaths', 'Population']].sum()

# Letra f - Crie uma nova coluna contendo o número de mortes por milhão dentro dos continentes
deaths_per_million = []
for i in range(covid19_world_continent.shape[0]):
    if pd.notna(covid19_world_continent['Population'])[i] == True and pd.notna(covid19_world_continent['Deaths'])[i] == True:
        deaths_per_million.append((covid19_world_continent['Deaths'][i] / covid19_world_continent['Population'][i])*(10**6))
    else:
        deaths_per_million.append(0)
    i += 1

covid19_world_continent['Deaths per million'] = deaths_per_million

# Letra g - Faça um ranking do número de mortes por milhão de habitantes entre os continentes
covid19_world_continent = covid19_world_continent.sort_values(by = 'Deaths per million', ascending = False)
print("\nRanking de número de mortes por milhão:")
covid19_world_continent = covid19_world_continent.reset_index()
i = 0
while i < covid19_world_continent.shape[0]:
    print('- Continente: {} - Mortes por milhão: {}'.format(
        covid19_world_continent["Continent"][i], 
        covid19_world_continent["Deaths per million"][i]))
    i += 1


Ranking de número de mortes por milhão:
- Continente: South America - Mortes por milhão: 314.3185130389227
- Continente: Europe - Mortes por milhão: 208.59393308569412
- Continente: North America - Mortes por milhão: 94.89002716925498
- Continente: Asia - Mortes por milhão: 27.016157008918057
- Continente: Africa - Mortes por milhão: 13.218306856453369
- Continente: Australia/Oceania - Mortes por milhão: 2.58802274305556
